# Intraguild Predation (IGP) model

\begin{align}
\dfrac{\partial R}{\partial t} &= D_R \dfrac{\partial^2 R}{\partial x^2} + rR\left(1-\dfrac{R}{K}\right) -\left( \dfrac{a_{CR} R}{1 +a_{CR} T_{hCR} R} \right)C -\left( \dfrac{a_{PR} R}{1 + a_{PR} T_{hPR} R + a_{PC} T_{hPC} C + a_{PA} T_{hPA} A} \right)P \\
\dfrac{\partial C}{\partial t} &= D_C \dfrac{\partial^2 C}{\partial x^2} + ef_{RC} \left( \dfrac{C}{\theta_C + C} \right) \left( \dfrac{a_{CR} R}{1 +a_{CR} T_{hCR} R} \right)C -\left( \dfrac{a_{PC} C}{1 + a_{PR} T_{hPR} R + a_{PC} T_{hPC} C + a_{PA} T_{hPA} A} \right)P - m_C C \\
\dfrac{\partial P}{\partial t} &= D_P \dfrac{\partial^2 P}{\partial x^2} + \left( \dfrac{ef_{RP} a_{PR} R + ef_{CP} a_{PC} C + ef_{AP} a_{PA} A}{1 + a_{PR} T_{hPR} R + a_{PC} T_{hPC} C + a_{PA} T_{hPA} A} \right)P - m_P P - q_P P^2
\end{align}

Retrieving data stored spaced by 1.0 as time-records:

## Neumann boundary conditions and $\theta=0$ (no Allee effects)

In [ ]:
from pde import (PDE, FieldCollection, PlotTracker, ScalarField, UnitGrid, 
                 CartesianGrid, MemoryStorage, ProgressTracker)
from pde import ExplicitSolver, ImplicitSolver, Controller, ScipySolver
import numpy as np
from numpy import linalg as LA

f = open("neumann_no_Allee.txt", "w") # output file

def CLV_PDE_solver(
    DC,
    A,
):
    """
    IGP implementation using py-pde package.
    """
    # Fixed parameters
    DR = 0.4
    DP = 0.5
    r = 3
    K = 1
    aCR = 5
    ThCR = 1.2
    efRC = 1
    thetaC = 0
    mC = 0.4
    aPC = 0.7
    aPR = 0.03
    aPA = 0.1
    ThPC = 2.5
    ThPR = 6
    ThPA = 2
    efRP = 1
    efCP = 1
    efAP = 1
    mP = 0.2
    qP = 0.4
    # Define the reactive part of the model
    f_function = f"+ {r} * R * ( 1 - R / {K} ) - ( {aCR} * R / ( 1 + {aCR} * {ThCR} * R ) ) * C - ( {aPR} * R / ( 1 + {aPR} * {ThPR} * R + {aPC} * {ThPC} * C + {aPA} * {ThPA} * {A} ) ) * P"
    g_function = f"+ {efRC} * (C / ({thetaC} + C) ) * ( {aCR} * R / ( 1 + {aCR} * {ThCR} * R ) ) * C - ( {aPC} * C / ( 1 + {aPR} * {ThPR} * R + {aPC} * {ThPC} * C + {aPA} * {ThPA} * {A} ) ) * P - {mC} * C"
    h_function = f"+ ( ( {efRP} * {aPR} * R + {efCP} * {aPC} * C + {efAP} * {aPA} * {A} ) / ( 1 + {aPR} * {ThPR} * R + {aPC} * {ThPC} * C + {aPA} * {ThPA} * {A} ) ) * P - {mP} * P - {qP} * P * P"
    # Neumann Boundary condition
    bc_left = {"derivative": 0.0}
    bc_right = {"derivative": 0.0}
    bc = [bc_left, bc_right]
    # Definition of PDE system
    eq = PDE(
        {
            "R": f"{DR} * laplace(R)" + f_function,
            "C": f"{DC} * laplace(C)" + g_function,
            "P": f"{DP} * laplace(P)" + h_function,
        },
        bc=bc
    )
    # Defining the mesh
    x_min, x_max = 0, 10 # spatial domain
    dx = 0.05 # spatial discretization
    num_points_in_x = int((x_max - x_min) / dx)
    grid = CartesianGrid(bounds=[[x_min, x_max]], shape=num_points_in_x)
    # Initialize state (Initial Conditions)
    R = ScalarField.from_expression(
        grid, 
        f"0.4 * exp( - ( (x - (( {x_min} + {x_max} ) / 2.0)) ** 2.0 ) / 2.0 * 2.0 ** 2.0 )",
        label="Prey"
    )
    C = ScalarField.from_expression(
        grid, 
        f"1.2",
        label="Intermediate consumer"
    )
    P = ScalarField.from_expression(
        grid, 
        f"0.3",
        label="Predator"
    )
    state = FieldCollection([R, C, P])  # state vector
    # Define time tracker to plot and animate
    x_axis_limits = (x_min, x_max)
    y_axis_limits = (0, 2)
    tracker_plot_config = PlotTracker(show=True, plot_args={
        'ax_style': {'xlim': x_axis_limits, 'ylim': y_axis_limits},
    })
    storage = MemoryStorage()
    dt = 1e-2 # time discretization
    trackers = [
        "progress",  # show progress bar during simulation
        "steady_state",  # abort if steady-state is reached
        storage.tracker(interval=1),  # store data every 1 simulation time units
        #tracker_plot_config,  # show images during simulation
    ]
    # Select backend solver
    solver = ScipySolver(eq, method='LSODA')  # SciPy solver
    # Setup solver
    tmax = 2000 # maximum simulation time, but the simulation will stop before if steady-state is reached
    controller = Controller(solver, t_range=[0, tmax], tracker=trackers)
    solve = controller.run(state, dt=dt)
    R_storage = storage.extract_field(0)
    C_storage = storage.extract_field(1)
    P_storage = storage.extract_field(2)
    tol = 1.0e-4 # tolerance for deciding if steady-state is reached comparing the integral of consecutive times
    # Calculate the integral over the spatial domain
    R_cell_volumes = R_storage.grid.cell_volumes
    R_integral = (R_storage.data[-1] * R_cell_volumes).sum() # integral of R over the spatial domain
    C_cell_volumes = C_storage.grid.cell_volumes
    C_integral = (C_storage.data[-1] * C_cell_volumes).sum() # integral of C over the spatial domain
    P_cell_volumes = P_storage.grid.cell_volumes
    P_integral = (P_storage.data[-1] * P_cell_volumes).sum() # integral of P over the spatial domain
    if R_integral <= 1.0e-5:
        behav = 0 # this means R extinction
    elif R_integral > 0:
        # Extract the 5 last integral values for comparison
        R_2 = (R_storage.data[-2] * R_cell_volumes).sum()
        R_3 = (R_storage.data[-3] * R_cell_volumes).sum()
        R_4 = (R_storage.data[-4] * R_cell_volumes).sum()
        R_5 = (R_storage.data[-5] * R_cell_volumes).sum()
        if LA.norm( R_integral - R_2 ) and LA.norm( R_integral - R_3 ) and LA.norm( R_integral - R_4 ) and LA.norm( R_integral - R_5 ) <= tol:
            behav = 2 # this means that the system reaches a stable equilibrium point
        t_recorded = storage.times # retrieve all recorded simulation times
        if t_recorded[-1] == tmax:
            behav = 1 # this means that the system reaches an unstable equilibrium point, but you can inscrease the tmax to check
    return solve, R_integral, C_integral, P_integral, DC, A, behav

itera=1
for DC in np.arange(0.5,4,0.5):
    for A in np.arange(1,20,1):
        EDP_solution, R_integral, C_integral, P_integral, DC, A, behav = CLV_PDE_solver(DC,A)
        f.write("R=" + str(R_integral) + "\t" + "C=" + str(C_integral) + "\t" + "P=" + str(P_integral) + "\t" + "DC=" + str(DC) + "\t" + "A=" + str(A) + "\t" + "behav=" + str(behav) + "\n")
        print("R=" + str(R_integral) + "\t" + "C=" + str(C_integral) + "\t" + "P=" + str(P_integral) + "\t" + "DC=" + str(DC) + "\t" + "A=" + str(A) + "\t" + "behav=" + str(behav) + "\t" + "\t" + "iteration:" + str(itera) + "\n")
        itera += 1
f.close()
exit()

## Neumann boundary conditions and $\theta=0.02$ (Allee effects acting on C)

In [ ]:
from pde import (PDE, FieldCollection, PlotTracker, ScalarField, UnitGrid, 
                 CartesianGrid, MemoryStorage, ProgressTracker)
from pde import ExplicitSolver, ImplicitSolver, Controller, ScipySolver
import numpy as np
from numpy import linalg as LA

f = open("neumann_Allee.txt", "w") # output file

def CLV_PDE_solver(
    DC,
    A,
):
    """
    IGP implementation using py-pde package.
    """
    # Fixed parameters
    DR = 0.4
    DP = 0.5
    r = 3
    K = 1
    aCR = 5
    ThCR = 1.2
    efRC = 1
    thetaC = 0.02
    mC = 0.4
    aPC = 0.7
    aPR = 0.03
    aPA = 0.1
    ThPC = 2.5
    ThPR = 6
    ThPA = 2
    efRP = 1
    efCP = 1
    efAP = 1
    mP = 0.2
    qP = 0.4
    # Define the reactive part of the model
    f_function = f"+ {r} * R * ( 1 - R / {K} ) - ( {aCR} * R / ( 1 + {aCR} * {ThCR} * R ) ) * C - ( {aPR} * R / ( 1 + {aPR} * {ThPR} * R + {aPC} * {ThPC} * C + {aPA} * {ThPA} * {A} ) ) * P"
    g_function = f"+ {efRC} * (C / ({thetaC} + C) ) * ( {aCR} * R / ( 1 + {aCR} * {ThCR} * R ) ) * C - ( {aPC} * C / ( 1 + {aPR} * {ThPR} * R + {aPC} * {ThPC} * C + {aPA} * {ThPA} * {A} ) ) * P - {mC} * C"
    h_function = f"+ ( ( {efRP} * {aPR} * R + {efCP} * {aPC} * C + {efAP} * {aPA} * {A} ) / ( 1 + {aPR} * {ThPR} * R + {aPC} * {ThPC} * C + {aPA} * {ThPA} * {A} ) ) * P - {mP} * P - {qP} * P * P"
    # Neumann Boundary condition
    bc_left = {"derivative": 0.0}
    bc_right = {"derivative": 0.0}
    bc = [bc_left, bc_right]
    # Definition of PDE system
    eq = PDE(
        {
            "R": f"{DR} * laplace(R)" + f_function,
            "C": f"{DC} * laplace(C)" + g_function,
            "P": f"{DP} * laplace(P)" + h_function,
        },
        bc=bc
    )
    # Defining the mesh
    x_min, x_max = 0, 10 # spatial domain
    dx = 0.05 # spatial discretization
    num_points_in_x = int((x_max - x_min) / dx)
    grid = CartesianGrid(bounds=[[x_min, x_max]], shape=num_points_in_x)
    # Initialize state (Initial Conditions)
    R = ScalarField.from_expression(
        grid, 
        f"0.4 * exp( - ( (x - (( {x_min} + {x_max} ) / 2.0)) ** 2.0 ) / 2.0 * 2.0 ** 2.0 )",
        label="Prey"
    )
    C = ScalarField.from_expression(
        grid, 
        f"1.2",
        label="Intermediate consumer"
    )
    P = ScalarField.from_expression(
        grid, 
        f"0.3",
        label="Predator"
    )
    state = FieldCollection([R, C, P])  # state vector
    # Define time tracker to plot and animate
    x_axis_limits = (x_min, x_max)
    y_axis_limits = (0, 2)
    tracker_plot_config = PlotTracker(show=True, plot_args={
        'ax_style': {'xlim': x_axis_limits, 'ylim': y_axis_limits},
    })
    storage = MemoryStorage()
    dt = 1e-2 # time discretization
    trackers = [
        "progress",  # show progress bar during simulation
        "steady_state",  # abort if steady state is reached
        storage.tracker(interval=1),  # store data every 1 simulation time units
        #tracker_plot_config,  # show images during simulation
    ]
    # Select backend solver
    solver = ScipySolver(eq, method='LSODA')  # SciPy solver
    # Setup solver
    tmax = 2000 # maximum simulation time, but the simulation will stop before if steady-state is reached
    controller = Controller(solver, t_range=[0, tmax], tracker=trackers)
    solve = controller.run(state, dt=dt)
    R_storage = storage.extract_field(0)
    C_storage = storage.extract_field(1)
    P_storage = storage.extract_field(2)
    tol = 1.0e-4 # tolerance for deciding if steady-state is reached comparing the integral of consecutive times
    # Calculate the integral over the spatial domain
    R_cell_volumes = R_storage.grid.cell_volumes
    R_integral = (R_storage.data[-1] * R_cell_volumes).sum() # integral of R over the spatial domain
    C_cell_volumes = C_storage.grid.cell_volumes
    C_integral = (C_storage.data[-1] * C_cell_volumes).sum() # integral of C over the spatial domain
    P_cell_volumes = P_storage.grid.cell_volumes
    P_integral = (P_storage.data[-1] * P_cell_volumes).sum() # integral of P over the spatial domain
    if R_integral <= 1.0e-5:
        behav = 0 # this means R extinction
    elif R_integral > 0:
        # Extract the 5 last integral values for comparison
        R_2 = (R_storage.data[-2] * R_cell_volumes).sum()
        R_3 = (R_storage.data[-3] * R_cell_volumes).sum()
        R_4 = (R_storage.data[-4] * R_cell_volumes).sum()
        R_5 = (R_storage.data[-5] * R_cell_volumes).sum()
        if LA.norm( R_integral - R_2 ) and LA.norm( R_integral - R_3 ) and LA.norm( R_integral - R_4 ) and LA.norm( R_integral - R_5 ) <= tol:
            behav = 2 # this means that the system reaches a stable equilibrium point
        t_recorded = storage.times # retrieve all recorded simulation times
        if t_recorded[-1] == tmax:
            behav = 1 # this means that the system reaches an unstable equilibrium point, but you can inscrease the tmax to check
    return solve, R_integral, C_integral, P_integral, DC, A, behav

itera=1
for DC in np.arange(0.5,4,0.5):
    for A in np.arange(1,20,1):
        EDP_solution, R_integral, C_integral, P_integral, DC, A, behav = CLV_PDE_solver(DC,A)
        f.write("R=" + str(R_integral) + "\t" + "C=" + str(C_integral) + "\t" + "P=" + str(P_integral) + "\t" + "DC=" + str(DC) + "\t" + "A=" + str(A) + "\t" + "behav=" + str(behav) + "\n")
        print("R=" + str(R_integral) + "\t" + "C=" + str(C_integral) + "\t" + "P=" + str(P_integral) + "\t" + "DC=" + str(DC) + "\t" + "A=" + str(A) + "\t" + "behav=" + str(behav) + "\t" + "\t" + "iteration:" + str(itera) + "\n")
        itera += 1
f.close()
exit()

## Dirichlet boundary conditions and $\theta=0$ (no Allee effects)

In [ ]:
from pde import (PDE, FieldCollection, PlotTracker, ScalarField, UnitGrid, 
                 CartesianGrid, MemoryStorage, ProgressTracker)
from pde import ExplicitSolver, ImplicitSolver, Controller, ScipySolver
import numpy as np
from numpy import linalg as LA

f = open("dirichlet_no_Allee.txt", "w") # output file

def CLV_PDE_solver(
    DC,
    A,
):
    """
    IGP implementation using py-pde package.
    """
    # Fixed parameters
    DR = 0.4
    DP = 0.5
    r = 3
    K = 1
    aCR = 5
    ThCR = 1.2
    efRC = 1
    thetaC = 0
    mC = 0.4
    aPC = 0.7
    aPR = 0.03
    aPA = 0.1
    ThPC = 2.5
    ThPR = 6
    ThPA = 2
    efRP = 1
    efCP = 1
    efAP = 1
    mP = 0.2
    qP = 0.4
    # Define the reactive part of the model
    f_function = f"+ {r} * R * ( 1 - R / {K} ) - ( {aCR} * R / ( 1 + {aCR} * {ThCR} * R ) ) * C - ( {aPR} * R / ( 1 + {aPR} * {ThPR} * R + {aPC} * {ThPC} * C + {aPA} * {ThPA} * {A} ) ) * P"
    g_function = f"+ {efRC} * (C / ({thetaC} + C) ) * ( {aCR} * R / ( 1 + {aCR} * {ThCR} * R ) ) * C - ( {aPC} * C / ( 1 + {aPR} * {ThPR} * R + {aPC} * {ThPC} * C + {aPA} * {ThPA} * {A} ) ) * P - {mC} * C"
    h_function = f"+ ( ( {efRP} * {aPR} * R + {efCP} * {aPC} * C + {efAP} * {aPA} * {A} ) / ( 1 + {aPR} * {ThPR} * R + {aPC} * {ThPC} * C + {aPA} * {ThPA} * {A} ) ) * P - {mP} * P - {qP} * P * P"
    # Dirichlet Boundary condition
    bc_left = {"value": 0.0}
    bc_right = {"value": 0.0}
    bc = [bc_left, bc_right]
    # Definition of PDE system
    eq = PDE(
        {
            "R": f"{DR} * laplace(R)" + f_function,
            "C": f"{DC} * laplace(C)" + g_function,
            "P": f"{DP} * laplace(P)" + h_function,
        },
        bc=bc
    )
    # Defining the mesh
    x_min, x_max = 0, 10 # spatial domain
    dx = 0.05 # spatial discretization
    num_points_in_x = int((x_max - x_min) / dx)
    grid = CartesianGrid(bounds=[[x_min, x_max]], shape=num_points_in_x)
    # Initialize state (Initial Conditions)
    R = ScalarField.from_expression(
        grid, 
        f"0.4 * exp( - ( (x - (( {x_min} + {x_max} ) / 2.0)) ** 2.0 ) / 2.0 * 2.0 ** 2.0 )",
        label="Prey"
    )
    C = ScalarField.from_expression(
        grid, 
        f"1.2",
        label="Intermediate consumer"
    )
    P = ScalarField.from_expression(
        grid, 
        f"0.3",
        label="Predator"
    )
    state = FieldCollection([R, C, P])  # state vector
    # Define time tracker to plot and animate
    x_axis_limits = (x_min, x_max)
    y_axis_limits = (0, 2)
    tracker_plot_config = PlotTracker(show=True, plot_args={
        'ax_style': {'xlim': x_axis_limits, 'ylim': y_axis_limits},
    })
    storage = MemoryStorage()
    dt = 1e-2 # time discretization
    trackers = [
        "progress",  # show progress bar during simulation
        "steady_state",  # abort if steady state is reached
        storage.tracker(interval=1),  # store data every 1 simulation time units
        #tracker_plot_config,  # show images during simulation
    ]
    # Select backend solver
    solver = ScipySolver(eq, method='LSODA')  # SciPy solve
    # Setup solver
    tmax = 2000 # maximum simulation time, but the simulation will stop before if steady-state is reached
    controller = Controller(solver, t_range=[0, tmax], tracker=trackers)
    solve = controller.run(state, dt=dt)
    R_storage = storage.extract_field(0)
    C_storage = storage.extract_field(1)
    P_storage = storage.extract_field(2)
    tol = 1.0e-4 # tolerance for deciding if steady-state is reached comparing the integral of consecutive times
    # Calculate the integral over the spatial domain
    R_cell_volumes = R_storage.grid.cell_volumes
    R_integral = (R_storage.data[-1] * R_cell_volumes).sum() # integral of R over the spatial domain
    C_cell_volumes = C_storage.grid.cell_volumes
    C_integral = (C_storage.data[-1] * C_cell_volumes).sum() # integral of C over the spatial domain
    P_cell_volumes = P_storage.grid.cell_volumes
    P_integral = (P_storage.data[-1] * P_cell_volumes).sum() # integral of P over the spatial domain
    if R_integral <= 1.0e-5:
        behav = 0 # this means R extinction
    elif R_integral > 0:
        # Extract the 5 last integral values for comparison
        R_2 = (R_storage.data[-2] * R_cell_volumes).sum()
        R_3 = (R_storage.data[-3] * R_cell_volumes).sum()
        R_4 = (R_storage.data[-4] * R_cell_volumes).sum()
        R_5 = (R_storage.data[-5] * R_cell_volumes).sum()
        if LA.norm( R_integral - R_2 ) and LA.norm( R_integral - R_3 ) and LA.norm( R_integral - R_4 ) and LA.norm( R_integral - R_5 ) <= tol:
            behav = 2 # this means that the system reaches a stable equilibrium point
        t_recorded = storage.times # retrieve all recorded simulation times
        if t_recorded[-1] == tmax:
            behav = 1 # this means that the system reaches an unstable equilibrium point, but you can inscrease the tmax to check
    return solve, R_integral, C_integral, P_integral, DC, A, behav

itera=1
for DC in np.arange(0.5,4,0.5):
    for A in np.arange(1,20,1):
        EDP_solution, R_integral, C_integral, P_integral, DC, A, behav = CLV_PDE_solver(DC,A)
        f.write("R=" + str(R_integral) + "\t" + "C=" + str(C_integral) + "\t" + "P=" + str(P_integral) + "\t" + "DC=" + str(DC) + "\t" + "A=" + str(A) + "\t" + "behav=" + str(behav) + "\n")
        print("R=" + str(R_integral) + "\t" + "C=" + str(C_integral) + "\t" + "P=" + str(P_integral) + "\t" + "DC=" + str(DC) + "\t" + "A=" + str(A) + "\t" + "behav=" + str(behav) + "\t" + "\t" + "iteration:" + str(itera) + "\n")
        itera += 1
f.close()

## Dirichlet boundary conditions and $\theta=0.02$ (Allee effects acting on C)

In [ ]:
from pde import (PDE, FieldCollection, PlotTracker, ScalarField, UnitGrid, 
                 CartesianGrid, MemoryStorage, ProgressTracker)
from pde import ExplicitSolver, ImplicitSolver, Controller, ScipySolver
import numpy as np
from numpy import linalg as LA

f = open("dirichlet_Allee.txt", "w") # output file

def CLV_PDE_solver(
    DC,
    A,
):
    """
    IGP implementation using py-pde package.
    """
    # Fixed parameters
    DR = 0.4
    DP = 0.5
    r = 3
    K = 1
    aCR = 5
    ThCR = 1.2
    efRC = 1
    thetaC = 0.02
    mC = 0.4
    aPC = 0.7
    aPR = 0.03
    aPA = 0.1
    ThPC = 2.5
    ThPR = 6
    ThPA = 2
    efRP = 1
    efCP = 1
    efAP = 1
    mP = 0.2
    qP = 0.4
    # Define the reactive part of the model
    f_function = f"+ {r} * R * ( 1 - R / {K} ) - ( {aCR} * R / ( 1 + {aCR} * {ThCR} * R ) ) * C - ( {aPR} * R / ( 1 + {aPR} * {ThPR} * R + {aPC} * {ThPC} * C + {aPA} * {ThPA} * {A} ) ) * P"
    g_function = f"+ {efRC} * (C / ({thetaC} + C) ) * ( {aCR} * R / ( 1 + {aCR} * {ThCR} * R ) ) * C - ( {aPC} * C / ( 1 + {aPR} * {ThPR} * R + {aPC} * {ThPC} * C + {aPA} * {ThPA} * {A} ) ) * P - {mC} * C"
    h_function = f"+ ( ( {efRP} * {aPR} * R + {efCP} * {aPC} * C + {efAP} * {aPA} * {A} ) / ( 1 + {aPR} * {ThPR} * R + {aPC} * {ThPC} * C + {aPA} * {ThPA} * {A} ) ) * P - {mP} * P - {qP} * P * P"
    # Dirichlet Boundary condition
    bc_left = {"value": 0.0}
    bc_right = {"value": 0.0}
    bc = [bc_left, bc_right]
    # Definition of PDE system
    eq = PDE(
        {
            "R": f"{DR} * laplace(R)" + f_function,
            "C": f"{DC} * laplace(C)" + g_function,
            "P": f"{DP} * laplace(P)" + h_function,
        },
        bc=bc  # comment here if you want to "free" the boundaries
    )
    # Defining the mesh
    x_min, x_max = 0, 10 # spatial domain
    dx = 0.05 # spatial discretization
    num_points_in_x = int((x_max - x_min) / dx)
    grid = CartesianGrid(bounds=[[x_min, x_max]], shape=num_points_in_x)
    # Initialize state (Initial Conditions)
    R = ScalarField.from_expression(
        grid, 
        f"0.4 * exp( - ( (x - (( {x_min} + {x_max} ) / 2.0)) ** 2.0 ) / 2.0 * 2.0 ** 2.0 )",
        label="Prey"
    )
    C = ScalarField.from_expression(
        grid, 
        f"1.2",
        label="Intermediate consumer"
    )
    P = ScalarField.from_expression(
        grid, 
        f"0.3",
        label="Predator"
    )
    state = FieldCollection([R, C, P])  # state vector
    # Define time tracker to plot and animate
    x_axis_limits = (x_min, x_max)
    y_axis_limits = (0, 2)
    tracker_plot_config = PlotTracker(show=True, plot_args={
        'ax_style': {'xlim': x_axis_limits, 'ylim': y_axis_limits},
    })
    storage = MemoryStorage()
    dt = 1e-2 # time discretization
    trackers = [
        "progress",  # show progress bar during simulation
        "steady_state",  # abort if steady state is reached
        storage.tracker(interval=1),  # store data every 5 simulation time units
        #tracker_plot_config,  # show images during simulation
    ]
    # Select backend solver
    solver = ScipySolver(eq, method='LSODA')  # SciPy solve
    # Setup solver
    tmax = 2000 # maximum simulation time, but the simulation will stop before if steady-state is reached
    controller = Controller(solver, t_range=[0, tmax], tracker=trackers)
    solve = controller.run(state, dt=dt)
    R_storage = storage.extract_field(0)
    C_storage = storage.extract_field(1)
    P_storage = storage.extract_field(2)
    tol = 1.0e-4 # tolerance for deciding if steady-state is reached comparing the integral of consecutive times
    # Calculate the integral over the spatial domain
    R_cell_volumes = R_storage.grid.cell_volumes
    R_integral = (R_storage.data[-1] * R_cell_volumes).sum() # integral of R over the spatial domain
    C_cell_volumes = C_storage.grid.cell_volumes
    C_integral = (C_storage.data[-1] * C_cell_volumes).sum() # integral of C over the spatial domain
    P_cell_volumes = P_storage.grid.cell_volumes
    P_integral = (P_storage.data[-1] * P_cell_volumes).sum() # integral of P over the spatial domain
    if R_integral <= 1.0e-5:
        behav = 0 # this means R extinction
    elif R_integral > 0:
        # Extract the 5 last integral values for comparison
        R_2 = (R_storage.data[-2] * R_cell_volumes).sum()
        R_3 = (R_storage.data[-3] * R_cell_volumes).sum()
        R_4 = (R_storage.data[-4] * R_cell_volumes).sum()
        R_5 = (R_storage.data[-5] * R_cell_volumes).sum()
        if LA.norm( R_integral - R_2 ) and LA.norm( R_integral - R_3 ) and LA.norm( R_integral - R_4 ) and LA.norm( R_integral - R_5 ) <= tol:
            behav = 2 # this means that the system reaches a stable equilibrium point
        t_recorded = storage.times # retrieve all recorded simulation times
        if t_recorded[-1] == tmax:
            behav = 1 # this means that the system reaches an unstable equilibrium point, but you can inscrease the tmax to check
    return solve, R_integral, C_integral, P_integral, DC, A, behav

itera=1
for DC in np.arange(0.5,4,0.5):
    for A in np.arange(1,20,1):
        EDP_solution, R_integral, C_integral, P_integral, DC, A, behav = CLV_PDE_solver(DC,A)
        f.write("R=" + str(R_integral) + "\t" + "C=" + str(C_integral) + "\t" + "P=" + str(P_integral) + "\t" + "DC=" + str(DC) + "\t" + "A=" + str(A) + "\t" + "behav=" + str(behav) + "\n")
        print("R=" + str(R_integral) + "\t" + "C=" + str(C_integral) + "\t" + "P=" + str(P_integral) + "\t" + "DC=" + str(DC) + "\t" + "A=" + str(A) + "\t" + "behav=" + str(behav) + "\t" + "\t" + "iteration:" + str(itera) + "\n")
        itera += 1
f.close()

## All parameters fixed for checking

In [ ]:
from pde import (PDE, FieldCollection, PlotTracker, ScalarField, UnitGrid, 
                 CartesianGrid, MemoryStorage, ProgressTracker)
from pde import ExplicitSolver, ImplicitSolver, Controller, ScipySolver
import numpy as np
from numpy import linalg as LA
from pde.storage import MemoryStorage
import matplotlib.pyplot as plt

f = open("checking.txt", "w") # output file

def CLV_PDE_solver(
    DC,
    A,
):
    """
    IGP implementation using py-pde package.
    """
    DR = 0.4
    DP = 0.5
    r = 3
    K = 1
    aCR = 5
    ThCR = 1.2
    efRC = 1
    thetaC = 0.02
    mC = 0.4
    aPC = 0.7
    aPR = 0.03
    aPA = 0.1
    ThPC = 2.5
    ThPR = 6
    ThPA = 2
    efRP = 1
    efCP = 1
    efAP = 1
    mP = 0.2
    qP = 0.4
    # Define the reactive part of the model
    f_function = f"+ {r} * R * ( 1 - R / {K} ) - ( {aCR} * R / ( 1 + {aCR} * {ThCR} * R ) ) * C - ( {aPR} * R / ( 1 + {aPR} * {ThPR} * R + {aPC} * {ThPC} * C + {aPA} * {ThPA} * {A} ) ) * P"
    g_function = f"+ {efRC} * (C / ({thetaC} + C) ) * ( {aCR} * R / ( 1 + {aCR} * {ThCR} * R ) ) * C - ( {aPC} * C / ( 1 + {aPR} * {ThPR} * R + {aPC} * {ThPC} * C + {aPA} * {ThPA} * {A} ) ) * P - {mC} * C"
    h_function = f"+ ( ( {efRP} * {aPR} * R + {efCP} * {aPC} * C + {efAP} * {aPA} * {A} ) / ( 1 + {aPR} * {ThPR} * R + {aPC} * {ThPC} * C + {aPA} * {ThPA} * {A} ) ) * P - {mP} * P - {qP} * P * P"
    # Dirichlet Boundary condition
    bc_left = {"value": 0.0} # if you want Neumann BC switch "value" by "derivative"
    bc_right = {"value": 0.0} # if you want Neumann BC switch "value" by "derivative"
    bc = [bc_left, bc_right]
    # Definition of PDE system
    eq = PDE(
        {
            "R": f"{DR} * laplace(R)" + f_function,
            "C": f"{DC} * laplace(C)" + g_function,
            "P": f"{DP} * laplace(P)" + h_function,
        },
        bc=bc  # comment here if you want to "free" the boundaries
    )
    # Defining the mesh
    x_min, x_max = 0, 10 # spatial domain
    dx = 0.05 # spatial discretization
    num_points_in_x = int((x_max - x_min) / dx)
    grid = CartesianGrid(bounds=[[x_min, x_max]], shape=num_points_in_x)
    # Initialize state (Initial Conditions)
    R = ScalarField.from_expression(
        grid, 
        f"0.4 * exp( - ( (x - (( {x_min} + {x_max} ) / 2.0)) ** 2.0 ) / 2.0 * 2.0 ** 2.0 )",
        label="Prey"
    )
    C = ScalarField.from_expression(
        grid, 
        f"1.2",
        label="Intermediate consumer"
    )
    P = ScalarField.from_expression(
        grid, 
        f"0.3",
        label="Predator"
    )
    state = FieldCollection([R, C, P])  # state vector
    # Define time tracker to plot and animate
    x_axis_limits = (x_min, x_max)
    y_axis_limits = (0, 2)
    tracker_plot_config = PlotTracker(show=True, plot_args={
        'ax_style': {'xlim': x_axis_limits, 'ylim': y_axis_limits},
    })
    storage = MemoryStorage()
    dt = 1e-2 # time discretization
    trackers = [
        "progress",  # show progress bar during simulation
        "steady_state",  # abort if steady state is reached
        storage.tracker(interval=1),  # store data every 5 simulation time units
        #tracker_plot_config,  # show images during simulation
    ]
    # Select backend solver
    solver = ScipySolver(eq, method='LSODA')  # SciPy solve
    # Setup solver
    tmax = 2000 # maximum simulation time, but the simulation will stop before if steady-state is reached
    controller = Controller(solver, t_range=[0, tmax], tracker=trackers)
    solve = controller.run(state, dt=dt)
    R_storage = storage.extract_field(0)
    C_storage = storage.extract_field(1)
    P_storage = storage.extract_field(2)
    tol = 1.0e-4 # tolerance for deciding if steady-state is reached comparing the integral of consecutive times
    # Calculate the integral over the spatial domain
    R_cell_volumes = R_storage.grid.cell_volumes
    R_integral = (R_storage.data[-1] * R_cell_volumes).sum() # integral of R over the spatial domain
    C_cell_volumes = C_storage.grid.cell_volumes
    C_integral = (C_storage.data[-1] * C_cell_volumes).sum() # integral of C over the spatial domain
    P_cell_volumes = P_storage.grid.cell_volumes
    P_integral = (P_storage.data[-1] * P_cell_volumes).sum() # integral of P over the spatial domain
    if R_integral <= 1.0e-5:
        behav = 0 # this means R extinction
    elif R_integral > 0:
        # Extract the 5 last integral values for comparison
        R_2 = (R_storage.data[-2] * R_cell_volumes).sum()
        R_3 = (R_storage.data[-3] * R_cell_volumes).sum()
        R_4 = (R_storage.data[-4] * R_cell_volumes).sum()
        R_5 = (R_storage.data[-5] * R_cell_volumes).sum()
        if LA.norm( R_integral - R_2 ) and LA.norm( R_integral - R_3 ) and LA.norm( R_integral - R_4 ) and LA.norm( R_integral - R_5 ) <= tol:
            behav = 2 # this means that the system reaches a stable equilibrium point
        else:
            behav = 1 # this means that the system reaches an unstable equilibrium point, but you can inscrease the tmax to check
    t_recorded = storage.times  # retrieve all recorded simulation times
    x_idx = 100 # choose a single point of the domain to plot
    R_storage_array = np.array(R_storage.data)  # convert list of arrays to a 2D array (to use 2D index)
    R_at_given_x = R_storage_array[:, x_idx]  # [t_idx, x_idx], retrieve all times (:) for a given x_idx
    f.write(str(R_at_given_x))
    f.close()
    plt.figure(figsize=(8, 6))
    plt.plot(t_recorded, R_at_given_x, "-x")
    plt.xlabel("Time")
    plt.ylabel(fr"$R$ at x = {R_storage.grid.axes_coords[0][x_idx]} and tmax = {t_recorded[-1]}")
    plt.grid()
    plt.show()
    return solve, R_integral, C_integral, P_integral, DC, A, behav

def find_idx_nearest_value_in_array(array, value): # auxiliary function to get the nearest available point of that we provide (x_idx)
    import numpy as np
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx, array[idx]

EDP_solution, R_integral, C_integral, P_integral, DC, A, behav = CLV_PDE_solver(2.2,10) # choose the values of DC and A to plot
print("R=" + str(R_integral) + "\t" + "C=" + str(C_integral) + "\t" + "P=" + str(P_integral) + "\t" + "DC=" + str(DC) + "\t" + "A=" + str(A) + "\t" + "behav=" + str(behav) + "\n")

# Spatially homogeneous IGP system

In [ ]:
#!python
from numpy import *
import pylab as p
from scipy import integrate

# Definition of parameters
r = 3
K = 1
aCR = 5
ThCR = 1.2
efRC = 1
thetaC = 0.02
mC = 0.4
aPC = 0.7
aPR = 0.03
aPA = 0.1
ThPC = 2.5
ThPR = 6
ThPA = 2
efRP = 1
efCP = 1
efAP = 1
mP = 0.2
qP = 0.4
A = 4

def dX_dt(X, t=0):
    return array([ r*X[0]*(1-X[0]/K)-(aCR*X[0]*X[1])/(1+aCR*ThCR*X[0])-(aPR*X[0]*X[2])/(1+aPR*ThPR*X[0]+aPC*ThPC*X[1]+aPA*ThPA*A) ,

efRC*(X[1]/(thetaC+X[1]))*(aCR*X[0]*X[1])/(1+aCR*ThCR*X[0])-(aPC*X[1]*X[2])/(1+aPR*ThPR*X[0]+aPC*ThPC*X[1]+aPA*ThPA*A)-mC*X[1] ,

(efRP*aPR*X[0]+efCP*aPC*X[1]+efAP*aPA*A)*X[2]/(1+aPR*ThPR*X[0]+aPC*ThPC*X[1]+aPA*ThPA*A)-mP*X[2]-qP*X[2]*X[2] ] )

t = linspace(0, 200,  1000)              # time
X0 = array([0.47, 1.21, 0.28])                     # initials conditions
X, infodict = integrate.odeint(dX_dt, X0, t, full_output=True)
infodict['message']                     # >>> 'Integration successful.'

#!python
pest, agent, IGpredator = X.T
f1 = p.figure()
p.plot(t, pest, 'y-', label='R')
p.plot(t, agent  , 'b-', label='C')
p.plot(t, IGpredator  , 'm-', label='P')
p.grid()
p.legend(loc='best')
p.xlabel('time')
p.ylabel('Population')
f1.savefig('evolution.png')